# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from mrcnn.config import Config
# import utils
from mrcnn import model as modellib, utils
from mrcnn import visualize
import yaml
from mrcnn.model import log
from PIL import Image

 
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Root directory of the project
ROOT_DIR = os.getcwd()
 
# ROOT_DIR = os.path.abspath("../")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
 
iter_num = 0
 
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class PlantsConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"
 
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # Number of classes (including background)
    NUM_CLASSES = 1 + 8  # background + 1 shapes
 
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 320
    IMAGE_MAX_DIM = 384
 
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8 * 6, 16 * 6, 32 * 6, 64 * 6, 128 * 6)  # anchor side in pixels
 
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 100
 
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
 
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50
 
 
config = PlantsConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  384
IMAGE_META_SIZE                21
IMAGE_MIN_DIM                  320
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [384 384   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES 

## Create Model and Load Trained Weights

In [3]:
class PlantsDataset(utils.Dataset):
    # 得到该图中有多少个实例（物体）
    def get_obj_index(self, image):
        n = np.max(image)
        return n
 
    # 解析labelme中得到的yaml文件，从而得到mask每一层对应的实例标签
    def from_yaml_get_class(self, image_id):
        info = self.image_info[image_id]
        with open(info['yaml_path']) as f:
            temp = yaml.load(f.read())
            labels = temp['label_names']
            del labels[0]
        return labels
 
    # 重新写draw_mask
    def draw_mask(self, num_obj, mask, image, image_id):
        # print("draw_mask-->",image_id)
        # print("self.image_info",self.image_info)
        info = self.image_info[image_id]
        # print("info-->",info)
        # print("info[width]----->",info['width'],"-info[height]--->",info['height'])
        for index in range(num_obj):
            for i in range(info['width']):
                for j in range(info['height']):
                    # print("image_id-->",image_id,"-i--->",i,"-j--->",j)
                    # print("info[width]----->",info['width'],"-info[height]--->",info['height'])
                    at_pixel = image.getpixel((i, j))
                    if at_pixel == index + 1:
                        mask[j, i, index] = 1
        return mask
 
    # 重新写load_shapes，里面包含自己的自己的类别
    # 并在self.image_info信息中添加了path、mask_path 、yaml_path
    # yaml_pathdataset_root_path = "/tongue_dateset/"
    # img_floder = dataset_root_path + "rgb"
    # mask_floder = dataset_root_path + "mask"
    # dataset_root_path = "/tongue_dateset/"
    def load_shapes(self, count, img_floder, mask_floder, imglist, dataset_root_path):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("shapes", 1, "xrz")
        self.add_class("shapes", 2, "hpl")
        self.add_class("shapes", 3, "lh")
        self.add_class("shapes", 4, "fgz")
        self.add_class("shapes", 5, "fcs")
        self.add_class("shapes", 6, "yjx")
        self.add_class("shapes", 7, "qnh")
        self.add_class("shapes", 8, "hl")
 
        for i in range(count):
            # 获取图片宽和高
            print(i)
            filestr = imglist[i].split(".")[0]
            # print(imglist[i],"-->",cv_img.shape[1],"--->",cv_img.shape[0])
            # print("id-->", i, " imglist[", i, "]-->", imglist[i],"filestr-->",filestr)
            # filestr = filestr.split("_")[1]
            mask_path = mask_floder + "/" + filestr + ".png"
            yaml_path = dataset_root_path + "labelme_json/" + filestr + "_json/info.yaml"
            print(dataset_root_path + "labelme_json/" + filestr + "_json/img.png")
            cv_img = cv2.imread(dataset_root_path + "labelme_json/" + filestr + "_json/img.png")
 
            self.add_image("shapes", image_id=i, path=img_floder + "/" + imglist[i],
                           width=cv_img.shape[1], height=cv_img.shape[0], mask_path=mask_path, yaml_path=yaml_path)
 
    # 重写load_mask
    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        global iter_num
        print("image_id", image_id)
        info = self.image_info[image_id]
        count = 1  # number of object
        img = Image.open(info['mask_path'])
        num_obj = self.get_obj_index(img)
        mask = np.zeros([info['height'], info['width'], num_obj], dtype=np.uint8)
        mask = self.draw_mask(num_obj, mask, img, image_id)
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count - 2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
 
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        labels = []
        labels = self.from_yaml_get_class(image_id)
        labels_form = []
        for i in range(len(labels)):
            if labels[i].find("xrz")!=-1:
                #print "baiheyu"
                labels_form.append("xrz")
            elif labels[i].find("hpl")!=-1:
                #print "guibeizhu"
                labels_form.append("hpl")
            elif labels[i].find("lh")!=-1:
                #print "hupilan"
                labels_form.append("lh")
            elif labels[i].find("fgz")!=-1:
                #print "luhui"
                labels_form.append("fgz")
            elif labels[i].find("fcs")!=-1:
                #print "moli"
                labels_form.append("fcs")
            elif labels[i].find("yjx")!=-1:
                #print "mudan"
                labels_form.append("yjx")
            elif labels[i].find("qnh")!=-1:
                #print "sanjiaomei"
                labels_form.append("qnh")
            elif labels[i].find("hl")!=-1:
                #print "xianrenzhang"
                labels_form.append("hl")
        class_ids = np.array([self.class_names.index(s) for s in labels_form])
        return mask, class_ids.astype(np.int32)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size * cols, size * rows))
    return ax

## Run Object Detection

In [ ]:
# 基础设置
dataset_root_path = "train_data/"
img_floder = dataset_root_path + "pic"
mask_floder = dataset_root_path + "cv2_mask"
# yaml_floder = dataset_root_path
imglist = os.listdir(img_floder)
count = len(imglist)
 
# train与val数据集准备
dataset_train = PlantsDataset()
dataset_train.load_shapes(count, img_floder, mask_floder, imglist, dataset_root_path)
dataset_train.prepare()
 
# print("dataset_train-->",dataset_train._image_ids)
 
dataset_val = PlantsDataset()
dataset_val.load_shapes(count, img_floder, mask_floder, imglist, dataset_root_path)
dataset_val.prepare()
 
# print("dataset_val-->",dataset_val._image_ids)
 
# Load and display random samples
# image_ids = np.random.choice(dataset_train.image_ids, 4)
# for image_id in image_ids:
#    image = dataset_train.load_image(image_id)
#    mask, class_ids = dataset_train.load_mask(image_id)
#    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
 
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
 
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last
 
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    # print(COCO_MODEL_PATH)
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)
 
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='heads')
 
# Fine tune all layers
# Passing layers="all" trains all layers. You can also
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=10,
            layers="all")

0
train_data/labelme_json/63_json/img.png
1
train_data/labelme_json/189_json/img.png
2
train_data/labelme_json/77_json/img.png
3
train_data/labelme_json/638_json/img.png
4
train_data/labelme_json/604_json/img.png
5
train_data/labelme_json/162_json/img.png
6
train_data/labelme_json/176_json/img.png
7
train_data/labelme_json/88_json/img.png
8
train_data/labelme_json/610_json/img.png
9
train_data/labelme_json/348_json/img.png
10
train_data/labelme_json/360_json/img.png
11
train_data/labelme_json/406_json/img.png
12
train_data/labelme_json/412_json/img.png
13
train_data/labelme_json/374_json/img.png
14
train_data/labelme_json/599_json/img.png
15
train_data/labelme_json/228_json/img.png
16
train_data/labelme_json/214_json/img.png
17
train_data/labelme_json/572_json/img.png
18
train_data/labelme_json/566_json/img.png
19
train_data/labelme_json/200_json/img.png
20
train_data/labelme_json/758_json/img.png
21
train_data/labelme_json/770_json/img.png
22
train_data/labelme_json/764_json/img.png
2

186
train_data/labelme_json/28_json/img.png
187
train_data/labelme_json/129_json/img.png
188
train_data/labelme_json/101_json/img.png
189
train_data/labelme_json/667_json/img.png
190
train_data/labelme_json/673_json/img.png
191
train_data/labelme_json/115_json/img.png
192
train_data/labelme_json/459_json/img.png
193
train_data/labelme_json/465_json/img.png
194
train_data/labelme_json/303_json/img.png
195
train_data/labelme_json/317_json/img.png
196
train_data/labelme_json/471_json/img.png
197
train_data/labelme_json/288_json/img.png
198
train_data/labelme_json/539_json/img.png
199
train_data/labelme_json/511_json/img.png
200
train_data/labelme_json/277_json/img.png
201
train_data/labelme_json/263_json/img.png
202
train_data/labelme_json/505_json/img.png
203
train_data/labelme_json/713_json/img.png
204
train_data/labelme_json/707_json/img.png
205
train_data/labelme_json/706_json/img.png
206
train_data/labelme_json/712_json/img.png
207
train_data/labelme_json/262_json/img.png
208
train_d

388
train_data/labelme_json/479_json/img.png
389
train_data/labelme_json/492_json/img.png
390
train_data/labelme_json/486_json/img.png
391
train_data/labelme_json/653_json/img.png
392
train_data/labelme_json/135_json/img.png
393
train_data/labelme_json/121_json/img.png
394
train_data/labelme_json/647_json/img.png
395
train_data/labelme_json/109_json/img.png
396
train_data/labelme_json/690_json/img.png
397
train_data/labelme_json/684_json/img.png
398
train_data/labelme_json/34_json/img.png
399
train_data/labelme_json/20_json/img.png
400
train_data/labelme_json/36_json/img.png
401
train_data/labelme_json/22_json/img.png
402
train_data/labelme_json/692_json/img.png
403
train_data/labelme_json/686_json/img.png
404
train_data/labelme_json/679_json/img.png
405
train_data/labelme_json/137_json/img.png
406
train_data/labelme_json/651_json/img.png
407
train_data/labelme_json/645_json/img.png
408
train_data/labelme_json/123_json/img.png
409
train_data/labelme_json/490_json/img.png
410
train_data

571
train_data/labelme_json/433_json/img.png
572
train_data/labelme_json/355_json/img.png
573
train_data/labelme_json/590_json/img.png
574
train_data/labelme_json/584_json/img.png
575
train_data/labelme_json/209_json/img.png
576
train_data/labelme_json/235_json/img.png
577
train_data/labelme_json/553_json/img.png
578
train_data/labelme_json/547_json/img.png
579
train_data/labelme_json/221_json/img.png
580
train_data/labelme_json/792_json/img.png
581
train_data/labelme_json/786_json/img.png
582
train_data/labelme_json/779_json/img.png
583
train_data/labelme_json/751_json/img.png
584
train_data/labelme_json/745_json/img.png
585
train_data/labelme_json/744_json/img.png
586
train_data/labelme_json/750_json/img.png
587
train_data/labelme_json/778_json/img.png
588
train_data/labelme_json/787_json/img.png
589
train_data/labelme_json/793_json/img.png
590
train_data/labelme_json/546_json/img.png
591
train_data/labelme_json/220_json/img.png
592
train_data/labelme_json/234_json/img.png
593
train_

760
train_data/labelme_json/556_json/img.png
761
train_data/labelme_json/230_json/img.png
762
train_data/labelme_json/224_json/img.png
763
train_data/labelme_json/542_json/img.png
764
train_data/labelme_json/218_json/img.png
765
train_data/labelme_json/797_json/img.png
766
train_data/labelme_json/783_json/img.png
767
train_data/labelme_json/754_json/img.png
768
train_data/labelme_json/740_json/img.png
769
train_data/labelme_json/768_json/img.png
770
train_data/labelme_json/769_json/img.png
771
train_data/labelme_json/741_json/img.png
772
train_data/labelme_json/755_json/img.png
773
train_data/labelme_json/782_json/img.png
774
train_data/labelme_json/796_json/img.png
775
train_data/labelme_json/219_json/img.png
776
train_data/labelme_json/225_json/img.png
777
train_data/labelme_json/543_json/img.png
778
train_data/labelme_json/557_json/img.png
779
train_data/labelme_json/231_json/img.png
780
train_data/labelme_json/580_json/img.png
781
train_data/labelme_json/594_json/img.png
782
train_

...

image_id 45


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # This is added back by InteractiveShellApp.init_path()


image_id 502
  1/100 [..............................] - ETA: 3765s - loss: 6.6189 - rpn_class_loss: 0.0296 - rpn_bbox_loss: 1.9381 - mrcnn_class_loss: 2.9345 - mrcnn_bbox_loss: 1.3439 - mrcnn_mask_loss: 0.3728

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # This is added back by InteractiveShellApp.init_path()


image_id 643
